Primer Trabajo FSI: Redes Neuronales

Importamos las librerías

In [1]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import numpy as np
import os
import kagglehub
from torchsummary import summary

C:\Users\Daniel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Como tenemos una carpeta con las imágenes y otra con las etiquetas (YOLO) lo primero que debemos hacer es modificar este formato para tener un fichero csv con las etiquetas.

In [2]:
# --- Configuración del Dataset de KaggleHub ---
# Reemplaza con el ID de tu dataset en KaggleHub. 
# Formato: 'owner/dataset-slug/version' o 'owner/dataset-slug'
KAGGLE_DATASET_ID = 'pkdarabi/cardetection' 

# 1. Descargar el dataset usando kagglehub
# Esto descargará y descomprimirá el dataset en una ubicación temporal/cache.
print(f"Descargando dataset: {KAGGLE_DATASET_ID}")
# 'download' devuelve la ruta local donde se guardó el dataset.
KAGGLE_DOWNLOAD_PATH = kagglehub.dataset_download(KAGGLE_DATASET_ID)
print(f"Dataset descargado en: {KAGGLE_DOWNLOAD_PATH}")

# --- Configuración de Rutas (Ajustadas a la descarga) ---
# **¡Ajuste crucial!** Reemplaza 'train/images' y 'train/labels' si tu dataset
# tiene una estructura de subcarpeta diferente (ej: 'images', 'labels' directamente).
# La mayoría de los datasets YOLO tienen una carpeta de 'train' o 'data'.

# Definición de las rutas finales
IMAGEN_DIR = os.path.join(KAGGLE_DOWNLOAD_PATH, 'car', 'train', 'images') 
ETIQUETAS_DIR = os.path.join(KAGGLE_DOWNLOAD_PATH, 'car', 'train', 'labels') 
CSV_SALIDA = 'yolo_labels_dataset.csv'
EXTENSION_IMAGEN = '.jpg'
EXTENSION_ETIQUETA = '.txt'

datos = []

print(f"\nEscaneando imágenes en: {IMAGEN_DIR}")

# 2. Iterar sobre los archivos de imagen para emparejar
if not os.path.exists(IMAGEN_DIR):
    print(f"ERROR: No se encontró el directorio de imágenes en {IMAGEN_DIR}. Revisa la estructura del dataset de KaggleHub.")
else:
    for archivo_imagen in os.listdir(IMAGEN_DIR):
        if archivo_imagen.lower().endswith(EXTENSION_IMAGEN):
            # El nombre base (sin extensión) es la clave de emparejamiento
            nombre_base = os.path.splitext(archivo_imagen)[0]
            
            # 3. Construir la ruta al archivo de etiqueta
            ruta_etiqueta = os.path.join(ETIQUETAS_DIR, nombre_base + EXTENSION_ETIQUETA)
            
            if os.path.exists(ruta_etiqueta):
                
                # 4. Leer el archivo de etiqueta
                with open(ruta_etiqueta, 'r') as f:
                    lineas = f.readlines()
                
                # 5. Procesar cada línea (cada objeto/bounding box)
                for linea in lineas:
                    partes = linea.strip().split()
                    
                    if len(partes) == 5:
                        # El primer valor es la CLASE (entero), el resto son coordenadas (flotantes)
                        clase_idx = int(partes[0])
                        x_center = float(partes[1])
                        y_center = float(partes[2])
                        width = float(partes[3])
                        height = float(partes[4])
                        
                        # 6. Almacenar los datos
                        datos.append({
                            'nombre_archivo': archivo_imagen,
                            'clase_indice': clase_idx,
                            'x_center': x_center,
                            'y_center': y_center,
                            'width': width,
                            'height': height
                        })
                    else:
                        print(f"¡Advertencia! Línea con formato incorrecto en {ruta_etiqueta}: {linea.strip()}")
            else:
                # Nota: Es común en detección que algunas imágenes no tengan objetos (no hay archivo .txt)
                # Si esto ocurre, la imagen no tendrá entradas en el CSV (lo cual es correcto).
                pass
                # print(f"¡Advertencia! No se encontró el archivo de etiqueta para: {archivo_imagen}")


# 7. Crear el DataFrame y guardarlo
df = pd.DataFrame(datos)
df.to_csv(CSV_SALIDA, index=False)

print("\n--- Resumen ---")
print(f"Total de cajas delimitadoras encontradas: {len(df)}")
if not df.empty:
    print("Conteo de objetos por clase (Índice):")
    print(df['clase_indice'].value_counts().sort_index())
print(f"¡CSV creado exitosamente en: {CSV_SALIDA}!")

Descargando dataset: pkdarabi/cardetection
Dataset descargado en: C:\Users\Daniel\.cache\kagglehub\datasets\pkdarabi\cardetection\versions\5

Escaneando imágenes en: C:\Users\Daniel\.cache\kagglehub\datasets\pkdarabi\cardetection\versions\5\car\train\images

--- Resumen ---
Total de cajas delimitadoras encontradas: 4298
Conteo de objetos por clase (Índice):
clase_indice
0     542
1     585
2      19
3     267
4     101
5     252
6     285
7     334
8     235
9     283
10    301
11    318
12    323
13    168
14    285
Name: count, dtype: int64
¡CSV creado exitosamente en: yolo_labels_dataset.csv!


Ahora ya podemos crear una instancia de la clase YOLODataset

In [3]:
# --- CÓDIGO PARA GENERAR EL DATASET ---
IMAGE_SIZE = (416, 416) 
DATA_DIR = './'
LABELS_NAME = 'yolo_labels_dataset.csv'

# 1. Definir transformaciones
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(), 
])


# 2. Clase para Detección de Objetos (SIN CAMBIOS)
class YOLODataset(Dataset):
    def __init__(self, archivo_csv, directorio_imagenes, transform=None):
        self.full_labels_df = pd.read_csv(archivo_csv)
        self.directorio_imagenes = directorio_imagenes
        self.transform = transform

        self.imagenes_unicas = self.full_labels_df['nombre_archivo'].unique()
        self.labels_grouped = self.full_labels_df.groupby('nombre_archivo')

    def __len__(self):
        return len(self.imagenes_unicas)

    def __getitem__(self, idx):
        image_name = self.imagenes_unicas[idx]
        image_path = os.path.join(self.directorio_imagenes, image_name)
        image = Image.open(image_path).convert('RGB')
        
        boxes_df = self.labels_grouped.get_group(image_name)
        
        labels_tensor = torch.tensor(
            boxes_df[['clase_indice', 'x_center', 'y_center', 'width', 'height']].values, 
            dtype=torch.float32
        )

        if self.transform:
            image = self.transform(image)

        return image, labels_tensor


# 2. Definir las rutas usando la ruta de descarga de Kaggle
ruta_csv = os.path.join(DATA_DIR, LABELS_NAME) # El CSV se creó en el directorio actual
# IMPORTANTE: Definir la ruta de imágenes APUNTANDO al subdirectorio 'train/images'
ruta_imgs = os.path.join(KAGGLE_DOWNLOAD_PATH, 'car', 'train', 'images') 

# 3. Crear instancia del Dataset
if not os.path.exists(ruta_csv) or not os.path.exists(ruta_imgs):
    print("Error: Asegúrate de que el CSV existe y que la ruta de imágenes de Kaggle es correcta.")
else:
    dataset_yolo = YOLODataset(archivo_csv=ruta_csv, directorio_imagenes=ruta_imgs, transform=transform)
    print(f"\nTipo de objeto creado: {type(dataset_yolo)}")
    print(f"Número total de imágenes (longitud del dataset): {len(dataset_yolo)}")

    # Ejemplo de acceso al primer elemento:
    if len(dataset_yolo) > 0:
        img_tensor, boxes_tensor = dataset_yolo[0]
        print(f"Forma del Tensor de la Imagen [0]: {img_tensor.shape}")
        print(f"Forma del Tensor de Cajas [0]: {boxes_tensor.shape}")
        print(f"Ejemplo de Cajas (primeras 3): \n{boxes_tensor[:3]}")


Tipo de objeto creado: <class '__main__.YOLODataset'>
Número total de imágenes (longitud del dataset): 3527
Forma del Tensor de la Imagen [0]: torch.Size([3, 416, 416])
Forma del Tensor de Cajas [0]: torch.Size([1, 5])
Ejemplo de Cajas (primeras 3): 
tensor([[7.0000, 0.5337, 0.3173, 0.1695, 0.3173]])


El siguiente paso será crear el DataLoader.

In [4]:
# --- Configuración del Loader ---
BATCH_SIZE = 16 # Tamaño de batch comúnmente usado en detección de objetos.

# 1. Definir la función de colación (collate_fn)
def yolo_collate_fn(batch):
    """
    Función de colación personalizada para detección de objetos.
    Agrupa las imágenes en un solo tensor y las etiquetas en una lista.
    """
    # Separar imágenes y etiquetas (cajas)
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch] # Cada target es un tensor [N_i, 5]

    # Apilar las imágenes en un único tensor [B, C, H, W]
    images_tensor = torch.stack(images, dim=0)
    
    # Devolver las etiquetas como una lista. La lista contiene los tensores de cajas
    # de tamaño variable, uno por imagen en el batch.
    targets_list = targets
    
    return images_tensor, targets_list

# 2. Crear el DataLoader
train_loader = DataLoader(
    dataset_yolo, # El dataset ya inicializado en el código anterior
    batch_size=BATCH_SIZE, 
    shuffle=True, # Barajar los datos para el entrenamiento
    collate_fn=yolo_collate_fn, # ¡Crucial para detección de objetos!
    num_workers=4 # Opcional: Para cargar datos más rápido
)

print(f"\nDataLoader creado con éxito. Número de batches: {len(train_loader)}")


DataLoader creado con éxito. Número de batches: 221


A continuación creamos la red neuronal que entrenaremos con el dataset que hemos creado.

In [5]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(416*416, 512) # 
        self.fc2 = nn.Linear(512, 128)    # Capa oculta con 128 neuronas
        self.fc3 = nn.Linear(128, 15)      # Capa de salida con 10 clases (0-14)
        self.activation = nn.Sigmoid()        # Función de activación Sigmoide
        self.softmax = nn.Softmax(dim=1)  # Función softmax para la capa de salida

    def forward(self, x):
        x = x.view(-1, 416*416)            # Aplanar la imagen de 416x416 a un vector de 173056
        #print(x.shape)                  # Mostrar la forma del tensor después de aplanarlo
        x = self.fc1(x)                
        x = self.activation(x)            # Función de activación Sigmoide en la capa oculta
        #print(x.shape)                   # Mostrar la forma del tensor después de la primera capa
        x = self.fc2(x)                  # Capa de salida
        x = self.activation(x)
        x = self.fc3(x)
        #print(x.shape)                   # Mostrar la forma del tensor después de la segunda capa
        x = self.softmax(x)              # Aplicar softmax para obtener probabilidades
        return x

Definimos la función de perdida y el optimizador.

In [7]:
model = SimpleNN()
summary(model, (3, 416, 416)) # Resumen del modelo
# El modelo y las dimension de entrad de los datos

criterion = nn.MSELoss() # Función de pérdida (Mean Squared Error)
optimizer = optim.SGD(
    model.parameters(),  # Parámetros del modelo
    lr=0.01 # ESTO ES MUY IMPORTANTE, LA TASA DE APRENDIZAJE!!!!!!!!!
) # Optimizador (Stochastic Gradient Descent)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 512]      88,605,184
           Sigmoid-2                  [-1, 512]               0
            Linear-3                  [-1, 128]          65,664
           Sigmoid-4                  [-1, 128]               0
            Linear-5                   [-1, 15]           1,935
           Softmax-6                   [-1, 15]               0
Total params: 88,672,783
Trainable params: 88,672,783
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.98
Forward/backward pass size (MB): 0.01
Params size (MB): 338.26
Estimated Total Size (MB): 340.25
----------------------------------------------------------------
